In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model  = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map = 'cuda',
    torch_dtype = "auto",
    trust_remote_code = True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.89s/it]


In [6]:
prompt = "Write an email apologizing to Sarah for tragic gardening mishap.Explain how it happened."

#tokenize the input
input_token_id = tokenizer(prompt,return_tensors='pt').input_ids.to('cuda')

#generate the text
generation_output = model.generate(
    input_ids = input_token_id,
    max_new_tokens = 20
)
print(tokenizer.decode(generation_output[0]))

Write an email apologizing to Sarah for tragic gardening mishap.Explain how it happened.



Dear Sarah,


I hope this message finds you well. I


In [7]:
print(input_token_id)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363, 25305,   293,
         16423,   292,   286,   728,   481, 29889,  9544,  7420,   920,   372,
          9559, 29889]], device='cuda:0')


In [8]:
for id in input_token_id[0]:
    print(tokenizer.decode(id))

Write
an
email
apolog
izing
to
Sarah
for
trag
ic
garden
ing
m
ish
ap
.
Exp
lain
how
it
happened
.


In [9]:
print(generation_output)

tensor([[14350,   385,  4876, 27746,  5281,   304, 19235,   363, 25305,   293,
         16423,   292,   286,   728,   481, 29889,  9544,  7420,   920,   372,
          9559, 29889,    13,    13,    13,    13, 29928,   799, 19235, 29892,
            13,    13,    13, 29902,  4966,   445,  2643, 14061,   366,  1532,
         29889,   306]], device='cuda:0')


In [12]:
tokenizer.decode([29889,29902,13])

'.I\n'

### Comparing Trained LLM Tokenizers

In [13]:
colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence,tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx,t in enumerate(token_ids):
        print(f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' ')

In [14]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [15]:
show_tokens(text,"bert-base-uncased")

c:\Users\swati\anaconda3\envs\env_LLMbook\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swati\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [16]:
show_tokens(text, "bert-base-cased")

c:\Users\swati\anaconda3\envs\env_LLMbook\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swati\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [17]:
show_tokens(text, "gpt2")

c:\Users\swati\anaconda3\envs\env_LLMbook\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swati\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [18]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

### Contextualized Word Embeddings From a Language Model

In [2]:
from transformers import AutoModel, AutoTokenizer

#load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
#load language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")
#tokenize sentence
tokens = tokenizer('Hello World',return_tensors='pt')
output = model(**tokens)[0]

c:\Users\swati\anaconda3\envs\env_LLMbook\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swati\.cache\huggingface\hub\models--microsoft--deberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\swati\anaconda3\envs\env_LLMbook\lib\site-packages\huggingface_hub\file_download.py:140: UserWa

In [3]:
output.shape

torch.Size([1, 4, 384])

In [4]:
for id in tokens['input_ids'][0]:
    print(tokenizer.decode(id))

[CLS]
Hello
 World
[SEP]


In [5]:
output

tensor([[[-3.2520,  0.1818, -0.1254,  ..., -0.0502, -0.2334,  0.8897],
         [-0.4673,  0.1730, -0.0206,  ..., -0.5289,  0.7303,  2.1177],
         [-0.4950,  0.0564,  0.2842,  ...,  1.0543, -0.1747,  1.3793],
         [-2.9601,  0.2129, -0.1138,  ...,  0.1518, -0.2094,  1.0494]]],
       grad_fn=<NativeLayerNormBackward0>)

### Text Embeddings (For Sentences and Whole Documents)

In [7]:
from sentence_transformers import SentenceTransformer

In [9]:
#Load model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
#convert text to text embeddings
vector = model.encode("Best movie ever!")

In [11]:
# Access the tokenizer used by the model
tokenizer = model.tokenizer

# Check the tokenizer type
print(type(tokenizer))

<class 'transformers.models.mpnet.tokenization_mpnet_fast.MPNetTokenizerFast'>


In [10]:
vector.shape

(768,)

### Word Embedding Beyond LLMs

In [13]:
import gensim.downloader as api

In [14]:
model = api.load("glove-wiki-gigaword-50")
model.most_similar([model['king']], topn=11)

[==================================================] 100.0% 66.0/66.0MB downloaded


[('king', 1.0000001192092896),
 ('prince', 0.8236179351806641),
 ('queen', 0.7839043140411377),
 ('ii', 0.7746230363845825),
 ('emperor', 0.7736247777938843),
 ('son', 0.766719400882721),
 ('uncle', 0.7627150416374207),
 ('kingdom', 0.7542161345481873),
 ('throne', 0.7539914846420288),
 ('brother', 0.7492411136627197),
 ('ruler', 0.7434253692626953)]